## **VISÃO COMPUTACIONAL - AULA 18**

**Objetivos da aula:**

*   apresentar o processo completo de visão estéreo
*   apresentar o conceito de disparidade e mapa de profundidade
*   praticar com reconstrução 3D

In [80]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from google.colab.patches import cv2_imshow

#### <b> CARGA DOS DADOS PARA RECONSTRUÇÃO 3D <b>


Esta sequencia de células é somente para descompactar o nosso arquivo de dados, formados pelas imagens e dados de calibração das câmeras. Em seguida, lemos os dados destes arquivos.



In [2]:
from zipfile import ZipFile
file_name = "data_road.zip"

with ZipFile(file_name, 'r') as zip:
  zip.extractall()

Done


In [88]:
data_folder_left = "data_road/training/image_2/"
data_folder_right = "data_road/training_right/image_3/"
data_folder_calib = "data_road/training/calib/"
cat = ['uu', 'uum', 'um']
IDX_LEN = 6

In [89]:
idx_num = 1
cat_idx = 2
fname = cat[cat_idx]+'_'+str(idx_num).zfill(IDX_LEN)
img_fname = fname + '.png'
calib_fname = fname + '.txt'

In [90]:
img_left_color = cv2.imread(data_folder_left + img_fname)
img_right_color = cv2.imread(data_folder_right + img_fname)

In [91]:
img_left_bw = cv2.blur(cv2.cvtColor(img_left_color, cv2.COLOR_RGB2GRAY),(5,5))
img_right_bw = cv2.blur(cv2.cvtColor(img_right_color, cv2.COLOR_RGB2GRAY),(5,5))

A imagem capturada pela câmera esquerda é mostrada abaixo:

In [ ]:
cv2_imshow(img_left_bw)

A imagem capturada pela câmera direita é mostrada abaixo:

In [ ]:
cv2_imshow(img_right_bw)

Abaixo, temos a leitura dos dados de calibração das câmeras:

In [ ]:

matrix_type_1 = 'P2'
matrix_type_2 = 'P3'

calib_file = data_folder_calib + calib_fname
with open(calib_file, 'r') as f:
    fin = f.readlines()
    for line in fin:
        if line[:2] == matrix_type_1:
            calib_matrix_1 = np.array(line[4:].strip().split(" ")).astype('float32').reshape(3,-1)
        elif line[:2] == matrix_type_2:
            calib_matrix_2 = np.array(line[4:].strip().split(" ")).astype('float32').reshape(3,-1)


#### <b> CÁLCULO DE DISPARIDADES </b>

A disparidade é qualquer mudança entre a posição de objetos
no par de imagens, e é inversamente proporcional à distância do
objeto até a lente da câmera. Portanto a distância pode ser
calculada obtendo a disparidade de um par estéreo, e a disparidade
pode ser estimada através da correspondência dos objetos nas
imagens.

A OpenCV disponibiliza uma implementação do Algoritmo de Emparelhamento de Blocos (Bock Matching Algorithm). 

Referência: https://en.wikipedia.org/wiki/Block-matching_algorithm



In [ ]:
stereo = cv2.StereoBM_create(numDisparities=96, blockSize=11)
disparity = stereo.compute(img_left_bw,img_right_bw)
img = disparity.copy()
cv2_imshow(img)

#### <b>EXERCÍCIO</b>

Um dos parâmetros de configuração para cálculo de disparidades é o numDisparities. O que este parâmetro controla ? Por que ele precisa ser múltiplo de 16 ?

In [ ]:
#faça seus experimentos aqui

#### <b>EXERCÍCIO</b>

O segundo parâmetro é o blockSize... o que controla a configuração deste parâmetro ?

In [ ]:
#faça seus experimentos aqui

#### <b> RECONSTRUÇÃO 3D </b>

A partir do mapa de disparidades, podemos obter a profundidade de cada ponto do par estéreo. Isto gera o chamado mapa de profundidade. 

<img src="https://www.researchgate.net/profile/Sing_Bing_Kang/publication/2313285/figure/fig1/AS:341573188505604@1458448802555/Relationship-between-the-baseline-b-disparity-d-focal-length-f-and-depth-z_W640.jpg"> </img>

Isto é feito pela função stereoRectify da OpenCV.

In [95]:
# Calculate depth-to-disparity
cam1 = calib_matrix_1[:,:3] # left image - P2
cam2 = calib_matrix_2[:,:3] # right image - P3

Tmat = np.array([0.54, 0., 0.])

rev_proj_matrix = np.zeros((4,4))

cv2.stereoRectify(cameraMatrix1 = cam1,cameraMatrix2 = cam2, \
                  distCoeffs1 = 0, distCoeffs2 = 0, \
                  imageSize = img_left_color.shape[:2], \
                  R = np.identity(3), T = Tmat, \
                  R1 = None, R2 = None, \
                  P1 =  None, P2 =  None, Q = rev_proj_matrix);


O resultado principal da stereoRectify é determinar a matriz inversa de projeção de uma das câmeras do setup de visão estéreo. Uma vez que tenhamos esta matriz, podemos reconstruir a estrutura 3D de qualquer objeto. 

In [ ]:
points = cv2.reprojectImageTo3D(img, rev_proj_matrix)

#reflect on x axis
reflect_matrix = np.identity(3)
reflect_matrix[0] *= -1
points = np.matmul(points,reflect_matrix)

#extract colors from image
colors = cv2.cvtColor(img_left_color, cv2.COLOR_BGR2RGB)

#filter by min disparity
mask = img > img.min()
out_points = points[mask]
out_colors = colors[mask]

#filter by dimension
idx = np.fabs(out_points[:,0]) < 4.5
out_points = out_points[idx]
out_colors = out_colors.reshape(-1, 3)
out_colors = out_colors[idx]

print("Pontos Reconstruídos:")
print(points[0])


#### <b> ARMAZENAMENTO DA NUVEM DE PONTOS </b>

A nuvem de pontos reconstruída anteriormente, pode ser armazenada de diversas maneiras. Uma das maneiras comuns é salvar os resultado no formato PLY.

Referência: https://en.wikipedia.org/wiki/PLY_(file_format)

In [ ]:
def write_ply(fn, verts, colors):
    ply_header = '''ply
    format ascii 1.0
    element vertex %(vert_num)d
    property float x
    property float y
    property float z
    property uchar red
    property uchar green
    property uchar blue
    end_header
    '''
    out_colors = colors.copy()
    verts = verts.reshape(-1, 3)
    verts = np.hstack([verts, out_colors])
    with open(fn, 'wb') as f:
        f.write((ply_header % dict(vert_num=len(verts))).encode('utf-8'))
        np.savetxt(f, verts, fmt='%f %f %f %d %d %d ')

write_ply('out.ply', out_points, out_colors)
print('%s saved' % 'out.ply')

#### <b> VERIFICAÇÃO DOS PONTOS PROJETADOS </b>

Uma vez que tenhamos reconstruídos os pontos, podemos verificar a qualidade da reconstruída, reprojetando os pontos reconstruídos para produzir uma nova imgame.

In [96]:
reflected_pts = np.matmul(out_points, reflect_matrix)
projected_img,_ = cv2.projectPoints(reflected_pts, np.identity(3), np.array([0., 0., 0.]), \
                          cam2[:3,:3], np.array([0., 0., 0., 0.]))
projected_img = projected_img.reshape(-1, 2)

In [99]:
blank_img = np.zeros(img_left_color.shape, 'uint8')
img_colors = img_right_color[mask][idx].reshape(-1,3)

for i, pt in enumerate(projected_img):
    pt_x = int(pt[0])
    pt_y = int(pt[1])
    if pt_x > 0 and pt_y > 0:
        # use the BGR format to match the original image type
        col = (int(img_colors[i, 2]), int(img_colors[i, 1]), int(img_colors[i, 0]))
        cv2.circle(blank_img, (pt_x, pt_y), 1, col)

In [ ]:
cv2_imshow(blank_img)

#### <b>EXERCÍCIO</b>

Como a geração do mapa de disparidades controla a qualidade dos pontos reconstruídos ? Faça experimentos para verificar como a qualidade do mapa de disparidades controla a qualidade da reconstrução.

In [ ]:
#faça seus experimentos aqui

#### <B> PRÓXIMOS EVENTOS </b>

<b> 24 NOV </b>  - REVISÃO PARA PROVA P2

<b> 26 NOV </b>  - PROVA P2

<b> 07 DEC </b>  - PROJETO II
